<a href="https://colab.research.google.com/github/Harshal292004/Harshal292004/blob/main/CreateDataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U -q transformers sentence-transformers bitsandbytes accelerate langchain_groq langgraph wikipedia  langchain-community langchain-experimental langchain-huggingface pypdf


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.1 MB/s eta 0:00:00


In [120]:
import os
from google.colab import drive
from langchain.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
# Standard library imports
import json
import asyncio
import requests
from dotenv import load_dotenv
from typing import List, Optional, Annotated,Dict
from typing_extensions import TypedDict

# Pydantic imports
from pydantic import BaseModel, Field, field_validator,validator


# LangChain imports
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage, ToolMessage
from langchain_core.runnables import RunnableLambda, chain as as_runnable, RunnableConfig
from langchain_core.tools import tool




# LangChain community imports
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_community.retrievers import WikipediaRetriever
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

# HuggingFace imports
from langchain_huggingface import  ChatHuggingFace,HuggingFaceEndpoint,HuggingFaceEmbeddings
from huggingface_hub import  login

# Groq imports
from langchain_groq import ChatGroq

# LangGraph imports
from langgraph.checkpoint.memory import MemorySaver
from langgraph.pregel import RetryPolicy
from langgraph.graph import END, StateGraph, START

# IPython Display
from IPython.display import Image, display

from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **USING  ChatGroq**

In [121]:
from langchain_groq import ChatGroq

llm=ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key="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",
)

# Some UTILITY FUNCTIONS

In [122]:
def recursively_get_pdf_files(directory):
    pdf_files = []
    try:
        if os.path.isfile(directory) and directory.endswith('.pdf'):
            pdf_files.append(directory)
        elif os.path.isdir(directory):
            for file in os.listdir(directory):
                full_path = os.path.join(directory, file)
                pdf_files.extend(recursively_get_pdf_files(full_path))
    except Exception as e:
        print(f"Error accessing {directory}: {e}")
    return pdf_files


In [1]:
def load_pdf(pdf_path):
  loader = PyPDFLoader(pdf_path)
  pdf_page_list=loader.load()
  return pdf_page_list

In [124]:
def load_pdfs(list_of_pdf_paths):
  pdf_list=[]
  for pdf_path in list_of_pdf_paths:
    pdf_list.append(load_pdf(pdf_path))
  return pdf_list

In [126]:
def get_embeddings_model(model_name="sentence-transformers/all-mpnet-base-v2",model_kwargs = {'device': 'cuda'},  encode_kwargs = {'normalize_embeddings': False}):
  embeddings_model = HuggingFaceEmbeddings(
      model_name=model_name,
      model_kwargs=model_kwargs,
      encode_kwargs=encode_kwargs
  )
  return embeddings_model

In [ ]:
embeddings_model= get_embeddings_model()

In [ ]:
text_spillter= SemanticChunker(embeddings_model)

In [ ]:
def split_pdf(pdf_page_list):
  pdf_spllited=text_spillter.split_documents(pdf_page_list)
  return pdf_spllited

In [2]:
def split_documents(pdf_list):
  splitted_pdf_list =[]
  for pdf in pdf_list:
    docs_spllited=split_pdf(pdf)
    splitted_pdf_list .append(docs_spllited)
  return splitted_pdf_list

# Agents

In [131]:
class ScoreDetails(BaseModel):

  score:float = Field(
      description="Score of the chunk or the paper"
  )
  justification:str = Field(
      description="Justification of the chunk or the paper"
  )

  @validator('score')
  def score_must_be_between_0_and_10(cls, v):
    if v < 0 or v > 10:
      raise ValueError('score must be between 0 and 10')
    return v


<ipython-input-131-7db4f4c3a788>:10: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator('score')


In [132]:
class PaperEvaluation(BaseModel):
  paper_id:str= Field(
      description="Unique identifier for the paper"
  )
  methodology:ScoreDetails= Field(
      description="Methodology score and justification of the chunk or the paper"
  )
  coherence:ScoreDetails = Field(
      description="Coherence score and justification of the chunk or the paper"
  )
  evidenceScore:ScoreDetails= Field(
      description="Evidence score and justification of the chunk or the paper"
  )
  technicalScore:ScoreDetails= Field(
      description="Technical score and justification of the chunk or the paper"
  )
  innovationScore:ScoreDetails= Field(
      description="Innovation score and justification of the chunk or the paper"
  )
  Publishable:bool = Field(
      description="Whether the paper is publishable or not"
  )
  justification:str = Field(
      description="Overall Justification for the classification of chunk or paper"
  )

In [76]:
gen_a_analysis_of_chunk = ChatPromptTemplate([
    (
        "system",
        "You are an intelligent assistant tasked with analyzing research papers to determine their publishability. Your role is to provide a detailed analysis of why a paper is labeled as publishable or non-publishable based on the quality of the paper."
    ),
    (
        "human",
        """
        You will evaluate the content of a semantic chunk of a research paper at a time,\
        as you have limitations on processing large texts.
        You will not be labeling the papers yourself,\
        but rather analyzing the provided labels based on previously labeled examples.
        Your analysis should focus on the following parameters:
        - Methodology
        - Coherence
        - Evidence
        - Techincal
        - Inovative

        You will be provided with the following information:
        - The content of the chunk: {content_of_chunk}
        - The publishability label of the paper: {publishable}
        - The unique identifier for the paper: {paper_id}

        Please provide a comprehensive analysis based on the above parameters, explaining why the paper has been labeled as it is.
        Score Should be strictly in the range 0 to  10 (fractions allowed)
    """
    )
])

In [118]:
gen_a_final_analysis_prompt = ChatPromptTemplate([
    (
        "system",
        "You are an intelligent assistant tasked with analyzing research papers to determine their publishability. Your role is to provide a detailed analysis of why a paper is labeled as publishable or non-publishable based on the quality of the paper."
    ),
    (
        "human",
        """
        You will evaluate a research paper based on predefined criteria that have been established beforehand.
        You will collaborate with an assistant researcher who has studied each chunk of the paper and provided scores and justifications.

        Your task is to:
        - Understand the provided scores as a list of information.
        - Determine the average score you want to assign to the paper.
        - Provide justifications for your score, negotiating with the assistant where necessary.

        You will be provided with a list of Pydantic models that contain the scores and justifications for each field.
        Use this information to judge the entire paper and provide an average assessment for each criterion.
        Note that you can give average score value between 0 and 10 (fractions allowed)
        {evaluations_summary}

        Based on the evaluations provided, please summarize the overall quality of the papers and provide any recommendations for improvement.
        """
    )
])


In [98]:
def analyze_paper(doc,paper_id,publishable=False):
  analysis_list=[]
  for d in doc:
    content=d.page_content
    if content=="" or len(content)<100:
      continue
    chunk_with_publishiabllity_and_paper_id = {
        "content_of_chunk": content,
        "publishable": publishable,
        "paper_id":paper_id
    }
    answer=gen_a_analysis_of_chunk_chain.invoke(chunk_with_publishiabllity_and_paper_id)
    analysis_list.append(answer)

  return analysis_list

# Develop a framework to label a single paper with just the path of the pdf and the publishablity

In [ ]:
import uuid

# Generate a random UUID
random_id = uuid.uuid4()
print(random_id)


In [3]:
def label_paper(path_of_pdf,publishable=False,paper_id=""):
  # load the pdf
  pages_loaded=load_pdf(path_of_pdf)
  # split the document semantically
  # build a framework where you can change on the type of splliting is used
  spllited_pdf=split_pdf(pages_loaded)

  #Now Kick in the Agentic workflow of actually labelling the papers
  #Analyze the chunks based on the evaluation schema predefined
  # either paper_id is provided or just create one :

  # Generate a random UUID
  if(paper_id==""):
    random_id = uuid.uuid4()
    paper_id=str(random_id)


  analysis_list=analyze_paper(spllited_pdf,publishable=publishable,paper_id=paper_id)

  #Now here we have got the justification and scoors on the predefined critarieas
  #We can also let user define the criteria or fall back to default

  #Now use the merge and devide for devicing the actuall papper labellings

  final_analysis = merge_and_create_summary_recurrsively(analysis_list, 0, len(analysis_list) - 1)

  return final_analysis


In [ ]:
#built a ui so that we can easily show case the results of labelliong a directory as well

def label_directory(directory='/content/drive/MyDrive/KDSH_2025_Dataset/Reference/Publishable',list_of_publisability=[True]):
  list_of_files=recursively_get_pdf_files('/content/drive/MyDrive/KDSH_2025_Dataset/Reference/Publishable')
  # for now consider all of them to be publoshable
  load_pdfs


In [123]:
list_of_files=recursively_get_pdf_files('/content/drive/MyDrive/KDSH_2025_Dataset/Reference/Publishable')

In [65]:
document_list= load_docs(list_of_files)

In [67]:
text_spillter= SemanticChunker(embeddings_model)

In [69]:
spllited_docs_list=split_documents(document_list)

In [129]:
def label_a_paper(pdf_path,paper_id,publishable=False):
  pass

In [130]:
def label_a_directory(directory):
  pass

In [74]:
chunk_with_publishiabllity_and_paper_id = {
    "content_of_chunk": content,
   "publishable": True,
    "paper_id": "R006"
 }

In [79]:
gen_a_analysis_of_chunk_chain= gen_a_analysis_of_chunk | llm.with_structured_output(PaperEvaluation)

In [80]:
prompt_value = gen_a_analysis_of_chunk.invoke(
    chunk_with_publishiabllity_and_paper_id
)

In [83]:
answer=gen_a_analysis_of_chunk_chain.invoke(chunk_with_publishiabllity_and_paper_id)

In [99]:
analysis_list=analyze_paper(spllited_docs_list[0],paper_id="R006",publishable=True)

In [101]:
len(analysis_list)

18

In [103]:
def print_list_beautifully(list_of_stuff):
  for index,stuff in enumerate(list_of_stuff):
    print("-"*20)
    print("Stuff Number " + str(index+1))
    print("-"*20)
    print(stuff)
    print("-"*20)

In [107]:
def get_analysis_summary(evaluations):
  formatted_evaluations = []
  for evaluation in evaluations:
      formatted_evaluations.append(
          f"Paper ID: {evaluation.paper_id}\n"
          f"Methodology Score: {evaluation.methodology.score} - Justification: {evaluation.methodology.justification}\n"
          f"Coherence Score: {evaluation.coherence.score} - Justification: {evaluation.coherence.justification}\n"
          f"Evidence Score: {evaluation.evidenceScore.score} - Justification: {evaluation.evidenceScore.justification}\n"
          f"Technical Score: {evaluation.technicalScore.score} - Justification: {evaluation.technicalScore.justification}\n"
          f"Innovation Score: {evaluation.innovationScore.score} - Justification: {evaluation.innovationScore.justification}\n"
          f"Publishable: {evaluation.Publishable}\n"
          f"Overall Justification: {evaluation.justification}\n"
          "----------------------------------------\n"
      )
  evaluations_summary = "\n".join(formatted_evaluations)
  return evaluations_summary

In [109]:
gen_a_final_analysis_chain=gen_a_final_analysis_prompt|llm.with_structured_output(PaperEvaluation)

In [113]:
def get_final_analysis(analysis_list):
  evaluations_summary = get_analysis_summary(analysis_list)
  final_analysis= gen_a_final_analysis_chain.invoke(evaluations_summary)
  return final_analysis

In [115]:
import time

def merge_and_create_summary_recurrsively(analysis_list, left, right, wait_time=2):
    """
    Recursively summarizes the analysis list using divide-and-conquer with a wait time between calls.

    Parameters:
    - analysis_list: List of analyses to be summarized.
    - left: Starting index of the current sublist.
    - right: Ending index of the current sublist.
    - wait_time: Time to wait (in seconds) between summarization calls.

    Returns:
    - Summarized analysis for the specified range.
    """
    # Base case: If the range is small enough, summarize directly
    if right - left + 1 <= 4:  # At most 4 items
        sublist = analysis_list[left:right + 1]
        time.sleep(wait_time)  # Introduce wait time before calling the model
        return get_final_analysis(sublist)

    # Recursive division
    mid = (left + right) // 2
    left_summary = merge_and_create_summary_recurrsively(analysis_list, left, mid, wait_time)
    time.sleep(wait_time)  # Wait before processing the next half
    right_summary = merge_and_create_summary_recurrsively(analysis_list, mid + 1, right, wait_time)

    # Combine the summaries of the left and right halves
    time.sleep(wait_time)  # Wait before combining summaries
    return get_final_analysis([left_summary, right_summary])


In [119]:
final_summary = merge_and_create_summary_recurrsively(analysis_list, 0, len(analysis_list) - 1)

ValidationError: 2 validation errors for PaperEvaluation
evidenceScore.score
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=7.67, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/int_from_float
technicalScore.score
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=8.67, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/int_from_float